<a href="https://colab.research.google.com/github/lcbjrrr/quantai/blob/main/J_Dec_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
versioninfo()

Julia Version 1.10.8
Commit 4c16ff44be8 (2025-01-22 10:06 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, broadwell)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [2]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
using CSV
using DataFrames

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [63]:
Pkg.add("MLJ")
using MLJ

   Resolving package versions...
   Installed EarlyStopping ──────────── v0.3.0
   Installed InitialValues ──────────── v0.3.1
   Installed CategoricalDistributions ─ v0.1.15
   Installed ShowCases ──────────────── v0.1.0
   Installed LearnAPI ───────────────── v0.1.0
   Installed Distances ──────────────── v0.10.12
   Installed Accessors ──────────────── v0.1.42
   Installed StatisticalTraits ──────── v3.4.0
   Installed IterationControl ───────── v0.5.4
   Installed PrettyPrint ────────────── v0.2.0
   Installed MLUtils ────────────────── v0.4.7
   Installed ContextVariablesX ──────── v0.1.3
   Installed MLCore ─────────────────── v1.0.0
   Installed FLoopsBase ─────────────── v0.1.1
   Installed MLJFlow ────────────────── v0.5.0
   Installed StatisticalMeasures ────── v0.1.7
   Installed MicroCollections ───────── v0.2.0
   Installed DefineSingletons ───────── v0.1.2
   Installed NameResolution ─────────── v0.1.5
   Installed ComputationalResources ─── v0.3.2
   Installed PrettyPrin

In [3]:
Pkg.add("DecisionTree")
using DecisionTree

   Resolving package versions...
   Installed ScikitLearnBase ─ v0.5.0
   Installed DecisionTree ──── v0.12.4
    Updating `~/.julia/environments/v1.10/Project.toml`
  [7806a523] + DecisionTree v0.12.4
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [7806a523] + DecisionTree v0.12.4
  [6e75b9c4] + ScikitLearnBase v0.5.0
Precompiling packages...
   1335.3 ms  ✓ ScikitLearnBase
   1424.8 ms  ✓ DecisionTree
  2 dependencies successfully precompiled in 4 seconds. 435 already precompiled.


In [109]:
Pkg.add("StatsBase")
using StatsBase

   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
⌅ [2913bbd2] + StatsBase v0.33.21
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
Precompiling packages...
   1671.7 ms  ? Metrics


In [106]:
Pkg.add("EvalMetrics")
using EvalMetrics

   Resolving package versions...
   Installed EvalMetrics ─ v0.3.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [251d5f9e] + EvalMetrics v0.3.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [251d5f9e] + EvalMetrics v0.3.0
Precompiling packages...
   1798.4 ms  ✓ MLDataDevices → MLDataDevicesFillArraysExt
   2652.3 ms  ✓ StructArrays → StructArraysGPUArraysCoreExt
   3361.1 ms  ✓ EvalMetrics
   2237.6 ms  ? Metrics
  3 dependencies successfully precompiled in 5 seconds. 512 already precompiled.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version
  1 dependencies failed but may be precompilable after restarting julia
  1 dependency had output during precompilation:
┌ Metrics
│  WARNING: Method definition onehot_encode(Any, Any) in module Metrics at /root/.julia/packages/Metrics/giLga/src/Classification.jl:5 overwritten on the same line (check for duplicate calls to `include`).
│  ERROR: Method overwriti

In [4]:
;wget https://raw.githubusercontent.com/lcbjrrr/quantai/refs/heads/main/datasets/car_insurance_fraud.csv

--2025-03-14 22:41:30--  https://raw.githubusercontent.com/lcbjrrr/quantai/refs/heads/main/datasets/car_insurance_fraud.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1480 (1.4K) [text/plain]
Saving to: ‘car_insurance_fraud.csv’

     0K .                                                     100% 17.7M=0s

2025-03-14 22:41:31 (17.7 MB/s) - ‘car_insurance_fraud.csv’ saved [1480/1480]



In [5]:
#read csv:
df=CSV.read("car_insurance_fraud.csv", DataFrame)
first(df,5)

Row,age,is_male,engine_size,months_last_claim,fraud
,Int64,Int64,Int64,Int64,Int64
1,71,0,120,24,1
2,40,0,70,12,1
3,78,0,200,12,0
4,31,1,120,6,1
5,32,1,120,12,0


In [69]:
train, test = MLJ.partition(df , 0.8)
test

Row,age,is_male,engine_size,months_last_claim,fraud
,Int64,Int64,Int64,Int64,Int64
1,82,1,200,6,0
2,37,0,120,12,0
3,29,0,200,24,0
4,35,0,120,12,0
5,69,1,70,24,0
6,26,1,70,12,1
7,28,1,70,12,1
8,67,0,200,12,0
9,78,1,200,24,0


In [70]:
tree = DecisionTreeClassifier(max_depth=3)
DecisionTree.fit!(tree,Matrix(train[:,[:age,:is_male,:engine_size,:months_last_claim]]),train[:,:fraud])

DecisionTreeClassifier
max_depth:                3
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [0, 1]
root:                     Decision Tree
Leaves: 8
Depth:  3

In [107]:
function accuracy(y,pred)
  acc = sum(pred.==y)/length(y)
  return acc
end
pred_train = DecisionTree.predict(tree,Matrix(train[:,[:age,:is_male,:engine_size,:months_last_claim]]) )
accuracy(train[:,:fraud],pred_train)*100

73.75

In [78]:
pred_test=DecisionTree.predict(tree,Matrix(test[:,[:age,:is_male,:engine_size,:months_last_claim]]) )
accuracy(test[:,:fraud],pred_test)*100

60.0

In [117]:
#Confusion Matrix
counts(test[:,:fraud],pred_test)

2×2 Matrix{Int64}:
 7  6
 2  5

In [115]:
EvalMetrics.precision(test[:,:fraud],pred_test)

0.45454545454545453

In [114]:
EvalMetrics.recall(test[:,:fraud],pred_test)

0.7142857142857143

![](https://pbs.twimg.com/media/GmCuR9AbEAA12KO?format=png&name=small)